## Model

In [ ]:
python model.py --model=linearRegression --split=0.1

### arguments 

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-m", "--model", help="choix du model",
                    action="store_true")
parser.add_argument("-s", "--split", help="choix du model",
                    type=float)
args = parser.parse_args()
args.model
args.split

# Classes

## DataHandler 

In [2]:
import pandas as pd 

class DataHandler:
    """
        recuperer les data du GCP bucket
    """
    def __inti__(self):
        self.df_listings_final = None
        self.df_price_availability = None
        self.df_merged = None

    def get_data(self):
        """
            charge les datasets des csv
        """
        self.df_listings_final = pd.read_csv('listings_final.csv', delimiter=";",index_col = 0)
        self.df_price_availability = pd.read_csv('price_availability.csv', delimiter=';')
        #self.df_listings_final = pd.read_csv('https://storage.googleapis.com/h3-data/listings_final.csv', delimiter=";",index_col = 0)
        #self.df_price_availability = pd.read_csv('phttps://storage.googleapis.com/h3-data/price_availability.csv', delimiter=';')
        print("Datasets chargés ")
        
    def group_data(self):
        """
            merge les 2 dataset sur la colonne listing_id
        """
        resultMeans = self.df_price_availability.groupby('listing_id').mean()
        resultMeans['listing_id'] = self.df_price_availability['listing_id'].unique()
        self.df_merged = pd.merge(self.df_listings_final, resultMeans[['available','local_price','min_nights']],on='listing_id')
        print("dataset merged")
        
    def get_process_data(self):
        """
            get_data() + group_data()
        """
        self.get_data()
        self.group_data()
        
        


### tests de la classe

In [3]:
data = DataHandler()

In [4]:
data.get_process_data()
%time 

Datasets chargés 
dataset merged
Wall time: 0 ns


## FeatureRecipe

In [18]:
import pandas as pd
import numpy as np
class FeatureRecipe:
    
    def __init__(self, data):
        self.df = data
        self.categories = []
        self.floats = []
        self.int = []
        self.drop = []
    
    def dropNaNPourcentage(self, seuil):
        """
            drop la colonne suivant un seuil %( param)
        """
        dimension = self.df.shape
        for colonne in self.df:
            nbNaN = self.df[colonne].isna().sum()
            pourcentage = (nbNaN / dimension[0]) * 100
            if pourcentage > seuil :
                del self.df[colonne]
                print('{} supprimée'.format(colonne))

    
    def dropUselessFeature(self):
        to_drop = ['is_rebookable','is_new_listing','is_fully_refundable','is_business_travel_ready']
        self.df.drop(columns=to_drop, inplace=True)
        for colonne in self.df:
            if self.df[colonne].nunique() <=1:
                print('{} supprimée'.format(colonne))
                del self.df[colonne]
    
    def dropDuplicates(self):
        """
            drop des duplicates de colonnes 
        """
        i=0
        for colonnes in self.df: 
            i+=1
            y=0
            for colonnesDuplicate in self.df: 
                y+=1
                if i!=y and (self.df[colonnes].equals(self.df[colonnesDuplicate])) == True:
                    del self.df[colonnesDuplicate]
                    print('colonne {} supprimée'.format(colonnesDuplicate))
    
    
    def separeteVariableTypes(self):
        print("separating columns")
        for colonne in self.df.columns:
            if self.df[colonne].dtypes == int:
                self.int.append(self.df[colonne])
            elif self.df[colonne].dtypes == float:
                self.floats.append(self.df[colonne])
            else:
                self.categories.append(self.df[colonne])
        print ("nombre de colonnes : {} \n \
            number of discreet values : {} \n \
            number of continuous values : {} \n \
            number of others : {} \n  \
            taille total : {}".format(len(self.df.columns),len(self.int), \
            len(self.floats),len(self.categories),len(self.int)+len(self.floats)+len(self.categories) ))
    
    def prepareData(self, seuil):
        self.dropUselessFeature()
        self.dropDuplicates()
        self.dropNaNPourcentage(seuil)
        self.separeteVariableTypes()

### tests de la classe

In [19]:
feature = FeatureRecipe(data.df_merged)
feature.get_duplicates()
%time 

0 1


ValueError: Can only compare identically-labeled DataFrame objects

## FeaturesExtractor

In [ ]:
import sklearn as skn
import matplotlib as plt
from sklearn.model_selection import train_test_split

class FeaturesExtractor:
    """
    Feature Extractor class
    """
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.x_train, self.x_test, self.y_train, self.y_test = None,None,None,None
        self.df = data
        self.flist = flist
    
    def extract(self):
        print("extraction ...")
        self.data.drop(columns=self.flist, inplace=True)
        print("extraction terminée")
        
    def split(self, test_size, rng, target):
        print("debut split")
        self.x_train, self.x_test, self.y_train, self.y_test\
        = train_test_split(self.df.loc[:,self.data])

## Model Builder

In [ ]:
class ModelBuilder:
    """
        Class for train and print results of ml model 
    """    
    def __init__(self, model_path: str = None, save: bool = None):
        pass    

    def __repr__(self):
        pass    
    def predict(self, X) -> np.ndarray:
        pass    
    def save_model(self, path:str):
        #with the format : 'model_{}_{}'.format(date)
        pass    
    def load_model(self):
        try:
            #load model
            pass
        except:
            pass

## creation fonctions 

In [29]:
import pandas as pd 
df_listings_final = pd.read_csv('listings_final.csv', delimiter=";",index_col = 0)
df_price_availability = pd.read_csv('price_availability.csv', delimiter=';')
resultMeans = df_price_availability.groupby('listing_id').mean()
resultMeans['listing_id'] = df_price_availability['listing_id'].unique()
               

In [81]:
df_merged.head(5)

,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor,available,local_price,min_nights
0,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,False,1.00,1.00,0.234536,45.948454,1.0
1,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,False,0.88,0.69,0.030075,44.473684,2.0
2,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,False,1.00,1.00,0.744624,49.000000,3.0
3,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,False,0.82,0.48,0.953125,165.000000,2.0
4,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,False,0.95,0.90,0.451128,69.774436,3.0
